# Black-Scholes-Merton model of European Option Pricing

In this notebook, I write three functions to calculate the price of an European option using the Black-Scholes-Merton model. 

The function bsm_analytic calculates the price of an European Call or Put Option using the analytic formula. The function bsm_simulation calculates the price using a Monte-Carlo simulation where we only simulate the distribution of stock prices on the expiry date. We can do this since the analytic solution of the SDE is known. The function bsm_sim_2 generates the entire stock price paths according to the Geometric Brownian Motion SDE using the Euler-Maruyama algorithm and then calculates the option price using the simulated stock prices on the expiry date.

These do not assume a dividend paying stock but the stock price can be adjusted by calculating the present value of the divident payments and by adjusting $d_1$ and $d_2$ suitably.

In [ ]:
import numpy as np
from scipy.stats import norm

## Analytic function:

The analytic form of the Black-Scholes-Merton formula is:

$\large C(S_0,T) = S_0 N(d_1) - K e^{-r_fT}N(d_2)$ (for Call)

and 

$\large C(S_0,T) = K e^{-r_fT}N(-d_2) - S_0 N(-d_1) $, (for Put)

while the function 

In the formula above $N(x)$ is the cumulative distribution formula for a standard, normal distribution.

$\large d_1 = \frac{ln(\frac{S_0}{K}) + (r_f + \frac{\sigma^2}{2})T}{\sigma \sqrt{T}}$ 

and $\large d_2 = d1 - \sigma \sqrt{T}$. 

here $S_0$ = stock price at time 0 <br/>
$K$ = strike price <br/>
$T$ = expiry <br/>
$\sigma$ = volatility <br/>
$r_f$ = risk-free rate <br/>

The function also takes an additional input called "Type" for the call/put option type. 

In [ ]:
def bsm_analytic(S_0,K,T,r_f,vol,Type):
    
    #inputs:
    #S_0: price at time 0
    #K: strike price
    #T: expiry time
    #r_f : risk-free rate
    #vol: volatility (in same units as T)
    #Type: Call/Put
    
    d_1 = (np.log(S_0/K) + (r_f + vol*vol/2.0)*T)/(vol*np.sqrt(T))
    
    d_2 = d_1 - vol*np.sqrt(T)
    
    if Type == 'Call':
        
        bsm_price = S_0*norm.cdf(d_1) - K*np.exp(-r_f*T)*norm.cdf(d_2)
        
    elif Type == 'Put':
        
        bsm_price = K*np.exp(-r_f*T)*norm.cdf(-d_2) - S_0*norm.cdf(-d_1)
        
    return bsm_price
    
    

In [ ]:
bsm_analytic(50.0, 45.0, 80/365, 0.02, 0.3, 'Call')

## Monte-Carlo Solution: 

The Monte Carlo solution is found by first simulating many stock price paths. The price of the option is simply the present value of the average payoff over all these paths. Since the payoff of an European option depends only on the stock price at maturity, we need not simulate the whole price path. We can only simulate the prices on the maturity date. The assumption of the Black-Scholes-Merton is that the stock price follows a Geometric Brownian Motion, i.e.

$\large \frac{dS}{S} = \mu dt + \sigma dW_t$

This equation is analytically solvable and it can be shown that:

$\large S(T) = S_0 e^{(\mu - \sigma^2/2)T + \sigma W_t}$

Hence, by only simulating a range of stock prices on the expiry date, we can calculate the price of the option. We can also simulate the entire price path using the analytic solution or the Euler-Maruyama method and for details of that see my module on GBM-stock-price. 

This function takes the following inputs:

$S_0$ = stock price at time 0 <br/>
$K$ = strike price <br/>
$T$ = expiry <br/>
$\sigma$ = volatility <br/>
$r_f$ = risk-free rate <br/>
Type = Call/Put for call or put option.

The function also takes an additional input called 'iterations' for which is the number of iterations we want to use for the averaging, the default is 10000.

In [4]:
def bsm_simulation(S_0,K,T,r_f,vol,Type,iterations = 100000):
    
    #inputs:
    #S_0: price at time 0
    #K: strike price
    #T: expiry time
    #r_f : risk-free rate
    #vol: volatility (in same units as T)
    #Type: Call/Put
    #iterations: number of iterations (default 10000)
    
    #stock price at time T
    
    S_T = S_0*np.exp(((r_f - vol**2.0/2.0)*T) + vol*np.sqrt(T)*np.random.normal(0.0,1.0,iterations))
    
    #obtaining the payoff of each iteration
    
    if Type == 'Call':
        option_exercise = S_T - K  
    elif Type == 'Put':
        option_exercise = K - S_T
    
    zeros = np.zeros(iterations)
    
    payoff = np.maximum(option_exercise, zeros)
    
    #discounting back at the risk free rate
    
    bsm_price = np.exp(-r_f*T)*np.mean(payoff)
    
    return bsm_price
    

In [5]:
bsm_simulation(50.0, 45.0, 80/365, 0.02, 0.3, 'Call')

6.017796011122605

### Timing each function

In [14]:
%timeit -n 1000 bsm_analytic(50.0, 45.0, 80/365, 0.02, 0.3, 'Call')

166 µs ± 15 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [15]:
%timeit -n 1000 bsm_simulation(50.0, 45.0, 80/365, 0.02, 0.3, 'Call')

3.18 ms ± 19.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


## Trying a different implement of the Monte Carlo simulation:

Here we will simulate the entire path of the stock price. Though the analytic form of the stock price saves us this step, it is actually important to generate the whole path since if the path is not analytically obtainable, like in the case of any interest rate derivative, the above function will not work.

In [67]:
def bsm_sim_2(S_0,K,T,r_f,vol,Type,iterations = 10000, n = 1000):
    
    #inputs:
    #S_0: price at time 0
    #K: strike price
    #T: expiry time
    #r_f : risk-free rate
    #vol: volatility (in same units as T)
    #Type: Call/Put
    #iterations: number of iterations (default 10000)
    #n = number of time steps (default 1000)
    
    #stock price at time T
    
    dt = T/n
    
    #The Equivalent of Risk-Neutral Pricing here is that the
    #stock on average grows at the risk free rate
    
    r_array = r_f*dt + vol*np.random.normal(0,1,(n,iterations))*np.sqrt(dt)
    
    price_array= np.zeros_like(r_array)
    
    price_array[0] = S_0

    #generating the stock price
    
    for t in range(1, n):
        price_array[t] = price_array[t-1] + price_array[t-1]*r_array[t]
        
    #stock price on expiry
    
    S_T = price_array[n-1]
    
    #calculating the option payoff 
    
    if Type == 'Call':
        option_exercise = S_T - K  
    elif Type == 'Put':
        option_exercise = K - S_T
       
    zeros = np.zeros(iterations)
    
    payoff = np.maximum(option_exercise, zeros)
    
    #discounting back at the risk free rate
    
    bsm_price = np.exp(-r_f*T)*np.mean(payoff)
        
    return bsm_price
    
    

In [68]:
bsm_sim_2(50.0, 45.0, 80/365, 0.02, 0.3, 'Call', iterations=10000)

6.004759591433664

In [69]:
%timeit -n 10 bsm_sim_2(50.0, 45.0, 80/365, 0.02, 0.3, 'Call')

341 ms ± 7.01 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
